11.2.2. Dirichlet Laplace Direct Method 
==========================
**keys**: homogeneous Dirichlet bvp, double and single layer potential, unknown Neumann data

In [ ]:
from netgen.occ import *
from ngsolve import *
from ngsolve.webgui import Draw
from ngsolve.bem import *
from ngsolve.solvers import CG

Consider the Dirichlet boundary value problem 

$$ \left\{ \begin{array}{rcl l} \Delta u &=& 0, \quad &\Omega \subset \mathbb R^3\,,\\ \gamma_0 u&=& u_0, \quad &\Gamma = \partial \Omega\,.\end{array} \right. $$ 

Let us choose the following ansatz for the solution $u\in H^1(\Omega)$ (direct ansatz) 

$$ u(x) = \underbrace{ \int\limits_\Gamma \displaystyle{\frac{1}{4\,\pi}\, \frac{1}{\| x-y\|} } \, u_1(y)\, \mathrm{d}\sigma_y}_{\displaystyle{ \mathrm{ SL}(u_1) }} - \underbrace{ \int\limits_\Gamma \displaystyle{\frac{1}{4\,\pi}\, \frac{\langle n(y) , x-y\rangle }{\| x-y\|^3} } \, u_0(y)\, \mathrm{d}\sigma_y}_{\displaystyle{ \mathrm{DL}(u_0) }}$$ 

and solve for the Neumann data $u_1 \in H^{-\frac12}(\Gamma)$ by the boundary element method, i.e., 

$$ \forall \, v\in H^{-\frac12}(\Gamma): \quad \left\langle \gamma_0 \left(\mathrm{SL}(u_1)\right), v \right\rangle_{-\frac12}= \left\langle u_0, v\right\rangle_{-\frac12} + \left\langle \gamma_0 \left(\mathrm{DL}(u_0)\right), v\right\rangle_{-\frac12}\,. $$

Define the geometry $\Omega \subset \mathbb R^3$ and create a mesh:

In [ ]:
sp = Sphere( (0,0,0), 1)
mesh = Mesh( OCCGeometry(sp).GenerateMesh(maxh=0.2)).Curve(4)

Create the finite element spaces for $H^{-\frac12}(\Gamma)$ and $H^{\frac12}(\Gamma)$ according to the given mesh:  

In [ ]:
fesL2 = SurfaceL2(mesh, order=3, dual_mapping=True)
u,v = fesL2.TnT()
fesH1 = H1(mesh, order=4)
uH1,vH1 = fesH1.TnT()
print ("ndofL2 = ", fesL2.ndof, "ndof H1 = ", fesH1.ndof)

Compute the interpolation of exact Dirichlet data $u_0$ in finite element space:

In [ ]:
uexa = 1/ sqrt( (x-1)**2 + (y-1)**2 + (z-1)**2 )
u0 = GridFunction(fesH1)
u0.Interpolate (uexa)
Draw (u0, mesh, draw_vol=False, order=3);

The discretisation of the above variational formulation leads to a system of linear equations, ie 

$$ \mathrm{V} \, \mathrm{u}_1 = \left( \frac12 \,\mathrm{M} + \mathrm{K} \right) \, \mathrm{u}_0\,, $$ 
where the linear operators are as follows
- $\mathrm{V}$ is the single layer operator. $\mathrm V$ is regular and symmetric.
- $\mathrm{M}$ is a mass matrix.
- $\mathrm{K}$ is the double layer operator. 

We approximate the linear operators as hmatrices and solve for the Neumann data $u_1$ with an iterative solver:

In [ ]:
u1 = GridFunction(fesL2)
pre = BilinearForm(u*v*ds, diagonal=True).Assemble().mat.Inverse()

with TaskManager():
    # V = SingleLayerPotentialOperator(fesL2, intorder=12)
    V = LaplaceSL(u*ds)*v*ds
    
    M = BilinearForm( uH1 * v.Trace() * ds(bonus_intorder=3)).Assemble()
    # K = DoubleLayerPotentialOperator(fesH1, fesL2, intorder=12)
    K = LaplaceDL(uH1*ds)*v*ds
    rhs = ( (0.5 * M.mat + K.mat)*u0.vec).Evaluate()
    CG(mat = V.mat, pre=pre, rhs = rhs, sol=u1.vec, tol=1e-8, maxsteps=50, initialize=False, printrates=True)

In [ ]:
Draw (u1, mesh, draw_vol=False, order=3);

Let's have a look at the exact Neumann data and compute the error of the numerical solution:

In [ ]:
graduexa = CF( (uexa.Diff(x), uexa.Diff(y), uexa.Diff(z)) )
n = specialcf.normal(3)
u1exa = graduexa*n
Draw (u1exa, mesh, draw_vol=False, order=3);
print ("L2-error =", sqrt (Integrate ( (u1exa-u1)**2, mesh, BND)))